In [ ]:
#!pip install simpletransformers
from config import *
import pandas as pd
from utils import *
import logging
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline 
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)



logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
data_dir='/content/drive/MyDrive/data_folder/data_with_15%_no_label.csv'
df = pd.read_csv(data_dir,encoding='utf-8')


input_data=df['Title_description']
targets =df[label_['label_list']]

In [ ]:
# set up ppreprocessing pipeline 
preprocessing_pipeline = Pipeline([
    ('remove_numbers', remove_numbers),
    ('remove_extra_spaces', remove_extra_spaces),
    ('remove_english_words', remove_english_words),
    ('remove_stopwords', remove_stopwords),
    ('remove_perenthesis', remove_perenthesis),
    ('remove_prone', remove_prone),
    ('remove_symbols', remove_symbols),
    ('remove_dash', remove_dash),
    ('remove_3rdbreket', remove_3rdbreket),
])

In [ ]:
#print 10 rows of targets with loc 
col_list=bert_label_maker(targets) 
#preprocessing the title_description 
input_data=preprocessing_pipeline.fit_transform(input_data)
# if your are having any problem with the preprocessing pipeline use preprocess_news function 
#input_data=preprocess_news(input_data)


df['lebel']=col_list 

new_df=pd.DataFrame({'Title_description':input_data,'lebel':col_list})
new_df=new_df.dropna()
new_df=new_df.reset_index(drop=True)
new_df.head(10)

,Title_description,lebel
0,১০ মাস ধরে উৎপাদন বন্ধ বিডি ওয়েল্ডিং অস্তিত্ব ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,এসইএমএল গ্রোথ ফান্ডের চাঁদা গ্রহণ চলছে: এসইএমএ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,জিএসপি ফাইনান্সের লভ্যাংশ ঘোষণা: ব্যাংক বহির্...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,যমুনা অয়েলের শেয়ারপ্রতি আয় ব্যাপক বেড়েছে: শেয়া...,"[0, 1, 1, 0, 0, 0, 0, 0, 0]"
4,৬ কোম্পানির আয়-ব্যয়ের হিসাব প্রকাশ: পুঁজিবাজার...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
5,৭ কোম্পানির আর্থিক প্রতিবেদন প্রকাশ: দ্বিতীয় প...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
6,ক্ষতিগ্রস্ত কোটার মেয়াদ বাড়লো ১ বছর: শেয়ারবিজন...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,জিকিউ বলপেনের শেয়ারদর বেড়েছে ৭.৫৫%: ঢাকা স্টক ...,"[1, 0, 1, 0, 0, 0, 0, 0, 0]"
8,মুনাফা কমেছে ফু-ওয়াং ফুডের: শেয়ারবাজারে তালিকা...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
9,ভালো লভ্যাংশের গুজবে রিপাবলিক ইন্স্যুরেন্সের দ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [ ]:

train, eval = train_test_split(new_df, test_size=split_config['test_size'], random_state=split_config['random_state'])   
train.shape

(8238, 2)

In [ ]:
model = MultiLabelClassificationModel(
     bert_model_config['model_type'], bert_model_config['model_name'],
    num_labels=num_labels,args=simple_transformer_config
  
)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassificatio

In [ ]:
model.train_model(train)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_dat

(2060, 0.07479254769577275)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


In [ ]:
print(result)
print(model_outputs)


{'LRAP': 0.978943304877883, 'eval_loss': 0.05170597892980209}
[[2.85530090e-03 9.96093750e-01 9.96093750e-01 ... 3.27301025e-03
  1.73377991e-03 4.38308716e-03]
 [6.71863556e-04 6.31332397e-04 1.09481812e-03 ... 3.24964523e-04
  5.48362732e-04 1.74140930e-03]
 [1.15585327e-03 1.83105469e-03 4.66537476e-03 ... 5.02014160e-03
  1.23023987e-03 9.96093750e-01]
 ...
 [9.44824219e-01 8.06427002e-03 9.97558594e-01 ... 3.55148315e-03
  5.34439087e-03 7.54928589e-03]
 [9.85839844e-01 4.77218628e-03 6.46209717e-03 ... 4.23049927e-03
  3.82423401e-03 4.48608398e-03]
 [4.65631485e-04 9.98535156e-01 1.11579895e-03 ... 6.43730164e-04
  4.10795212e-04 7.88688660e-04]]


In [ ]:
news="রেইসের ফান্ডের লভ্যাংশ ঘোষণা: পুঁজিবাজারের তালিকাভুক্ত মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা ট্রাস্টি। গত রোববার ফান্ডগুলোর ট্রাস্টি সভায় জুন সমাপ্ত হিসাব বছরের লভ্যাংশ ঘোষণা হয়। ফান্ডের লভ্যাংশের রেকর্ড ডেট ঘোষণা সেপ্টেম্বর।এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল, ইবিএল এনআরবি মিউচ্যুয়াল ফান্ড, পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ড, পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ড, ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ড, ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ড ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা হয়েছে। ফান্ডগুলোর ফান্ডের ইউনিট হোল্ডারদের শতাংশ, তিনটির শতাংশ, দুটির শতাংশ চারটি ফান্ডের ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা ট্রাস্টি। পুরোটাই রি-ইনভেস্টমেন্ট ইউনিট।তথ্য মতে, এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল এনআরবি মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা।ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।"

#preprocess news
news= preprocess_news(news)

#preprocess news with pipeline 



In [ ]:
predictions, raw_outputs = model.predict(["রেইসের ফান্ডের লভ্যাংশ ঘোষণা: পুঁজিবাজারের তালিকাভুক্ত মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা ট্রাস্টি। গত রোববার ফান্ডগুলোর ট্রাস্টি সভায় জুন সমাপ্ত হিসাব বছরের লভ্যাংশ ঘোষণা হয়। ফান্ডের লভ্যাংশের রেকর্ড ডেট ঘোষণা সেপ্টেম্বর।এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল, ইবিএল এনআরবি মিউচ্যুয়াল ফান্ড, পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ড, পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ড, ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ড, ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ড ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা হয়েছে। ফান্ডগুলোর ফান্ডের ইউনিট হোল্ডারদের শতাংশ, তিনটির শতাংশ, দুটির শতাংশ চারটি ফান্ডের ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা ট্রাস্টি। পুরোটাই রি-ইনভেস্টমেন্ট ইউনিট।তথ্য মতে, এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল এনআরবি মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা।ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।"])

In [ ]:
predictions

[[0, 0, 0, 0, 0, 0, 0, 0, 1]]